In [ ]:
from pandas import read_csv, DataFrame
from numpy import arange
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as tck
import matplotlib.lines as mlines
import seaborn as sns
from json import loads
sns.set_theme()

In [ ]:
plt.rcParams.update({
        'font.family': 'serif',
        'font.serif': [],
        'font.size' : 11,
        'text.usetex': True,
        'pgf.texsystem': 'pdflatex',
        'axes.facecolor':'white',
        'axes.edgecolor': 'lightgrey',
        'hatch.color': 'white',
        #'ytick.color': 'grey'
        })

MEDIUM_SIZE = 14
BIGGER_SIZE = 16
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize

## PTP Plot

In [ ]:
json = []
with open('ptp4l_2.json', 'r') as file:
    for line in file.readlines():
        json.append(loads(line))
n = 0
y = []
for i in json:
    m = i['MESSAGE']
    if "rms" in m:
        v = int(m.rsplit("rms")[1].rsplit("max")[0].strip())
        y.append(v)

x = arange(0, len(y), 1)

In [ ]:
colors = [c for c in sns.color_palette()]
fig, ax = plt.subplots()
plt.boxplot(y, patch_artist=True, boxprops=dict(facecolor=colors[0], hatch="//"))
"""
plt.grid(visible=True,
         which='major',
         color='lightgrey',
         linestyle='-')
plt.grid(visible=True,
         which='minor',
         color='lightgrey',
         linestyle='--')
"""
plt.xticks([])
ax.yaxis.set_minor_locator(tck.AutoMinorLocator())
plt.ylabel("Deviation rms [ns]")
plt.ylim([-0.5, 27])
plt.legend(handles=[mpatches.Patch(facecolor=colors[0],
                                label='GM Deviation',
                                hatch="//",
                                edgecolor="black"),
                    mlines.Line2D([0], [0], 
                                  marker='o',
                                  color='black',
                                  label='Outlier',
                                  markerfacecolor='None',
                                  markersize=7,
                                  linestyle='None'),
                    mlines.Line2D([0], [0],
                                  marker="None",
                                  color=colors[1],
                                  label='Median')
                   ])
#plt.show()
plt.savefig("ptp_accuracy.pdf", dpi=300)
plt.close()

In [ ]:
df = DataFrame(y, columns=["Deviation"])

In [ ]:
pkt_count=[]
stats_df = df.groupby('Deviation')['Deviation'].agg('count').pipe(DataFrame) \
            .rename(columns = {'Deviation': 'frequency'})
stats_df['pmf'] = stats_df['frequency'] / sum(stats_df['frequency'])
pkt_count.append(sum(stats_df['frequency']))
stats_df['cdf'] = stats_df['pmf'].cumsum()
stats_df['ccdf'] = 1 - stats_df['pmf'].cumsum()
stats_df.iloc[-1, stats_df.columns.get_loc('ccdf')] = 0
stats_df = stats_df.reset_index()

In [ ]:
plt.plot(stats_df['Deviation'], stats_df['cdf'], '--o')
#plt.plot(stats_dfs[1]['Latency (ns)'], stats_dfs[1]['ccdf'], '-.')
#plt.plot(stats_dfs[2]['Latency (ns)'], stats_dfs[2]['ccdf'], ':')

colors = [c for c in sns.color_palette()]

legend = mlines.Line2D([], [], color=colors[0], linestyle='--',
                          markersize=7, label='GM Deviation', marker='o')
plt.legend(handles=[legend],
           loc='right')

plt.grid(visible=True,
         which='major',
         color='lightgrey',
         linestyle='-')
plt.grid(visible=True,
         which='minor',
         color='lightgrey',
         linestyle='--')

#plt.yscale("log")
#plt.xscale("log")
#plt.ylim((3e-5,1.3))
#plt.xlim((3e3,3e6))

plt.xlabel("Deviation rms [ns]")
plt.ylabel("CDF")

plt.tight_layout()
plt.savefig("ptp_accuracy_cdf.png", dpi=300)
plt.close("all")
#plt.show()

## Other Plots

In [ ]:
INPUT = "-BBB"
FRAMESIZE = 60
TAS_CONFIG = 1
STREAMS = ["robotic", "audio", "video"]

configs = ['FibroLAN-FalconRX{}_No-QoS_{}_0.0percent'.format(INPUT, FRAMESIZE),
           'FibroLAN-FalconRX{}_No-QoS_{}_120.0percent'.format(INPUT, FRAMESIZE),
           'FibroLAN-FalconRX{}_IEEE-802.1p_{}_120.0percent'.format(INPUT, FRAMESIZE),
           'FibroLAN-FalconRX{}_TAS{}_{}_120.0percent'.format(INPUT, TAS_CONFIG, FRAMESIZE)]
#rel_path = '/home/arch/testbed_src/src/measurements/successful_measurements/'
rel_path = "/mnt/harddrive/data/backup/old/"


dfs = {}
for c in configs:
    dfs[c] = {}
    for s in STREAMS:
        dfs[c][s] = read_csv('{}{}/csv/{}.csv'.format(rel_path, c, s),
                             delimiter=",",
                             usecols=['Latency (ns)'],
                             dtype=int)

data = []

for c in configs:
    d = []
    for s in STREAMS:
        d.append(dfs[c][s]['Latency (ns)'])
    data.append(d)

labels = ['Baseline',
          'Baseline + BG',
          'IEEE 802.1Q + BG',
          'IEEE 802.1Qbv + BG']

In [ ]:
showfliers = True
notch = True
sym = 'o'
markersize = 0.5
colors = [c for c in sns.color_palette()]

i = 1
for c in data:
    box1 = plt.boxplot(c[0],
                       positions=[i-0.2],
                       showfliers=showfliers,
                       sym=sym,
                       patch_artist=True,
                       boxprops=dict(facecolor=colors[0], hatch="//"),
                       notch=notch,
                       flierprops=dict(markersize=markersize))
    box2 = plt.boxplot(c[1],
                       positions=[i],
                       showfliers=showfliers,
                       sym=sym,
                       patch_artist=True,
                       boxprops=dict(facecolor=colors[1], hatch="\\\\"),
                       notch=notch,
                       flierprops=dict(markersize=markersize))
    box3 = plt.boxplot(c[2],
                       positions=[i+0.2],
                       showfliers=showfliers,
                       sym=sym,
                       patch_artist=True,
                       boxprops=dict(facecolor=colors[2], hatch="oo"),
                       notch=notch,
                       flierprops=dict(markersize=markersize))
    i += 1.5

xlocs = arange(1, i, 1.5)

robotic_legend = mpatches.Patch(facecolor=colors[0],
                                label='Tactile',
                                hatch="//",
                                edgecolor="black")
audio_legend = mpatches.Patch(facecolor=colors[1],
                              label='Audio',
                              hatch="\\\\",
                              edgecolor="black")
video_legend = mpatches.Patch(facecolor=colors[2],
                              label='Video',
                              hatch="oo",
                              edgecolor="black")

plt.legend(handles=[robotic_legend,
                    audio_legend,
                    video_legend])
plt.xticks(xlocs, labels, rotation=15)
plt.ylabel('One-way Latency [ns]')
plt.yscale('log')
plt.grid(visible=True,
         which='major',
         color='lightgrey',
         linestyle='-')
plt.grid(visible=True,
         which='minor',
         color='lightgrey',
         linestyle='--')
plt.tight_layout()
plt.savefig('figure6.png',
            dpi=600)
plt.close()
#plt.show()